# HW1.1: Dictionary-based Tokenization


In this exercise, you are to implement a dictionary-based word segmentation algorithm. There are two Python functions that you need to complete:
<br>
* maximal_matching
* backtrack
</br>

Also, you have to find how to use word_tokenize() in PythaiNLP along with customer_dict by yourselves.

In [8]:
!uv add pythainlp
!uv add marisa_trie
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import get_corpus
from marisa_trie import Trie


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Part 1) Maximal Matching from PythaiNLP

### Create a toy dictionary to test the algorithm

This is based on the example shown in the lecture.
You will tokenize the following text string: "ไปหามเหสี!"
The toy dictionary provided in this exercise includes all the charaters, syllables, and words that appear that the text string.

In [39]:
thai_vocab = ["ไ","ป","ห","า","ม","เ","ห","ส","ี","ไป","หา","หาม","เห","สี","มเหสี","!"]
input_text = "ไปหามเหสี!"

### Example Dictionary

Write the `word_tokenize` function of PyThaiNLP with a custom dictionary above and using:
1. Longest matching algorithm `longest`
2. Maximal-matching algorithm `newmm`

Study `word_tokenize()` from PythaiNLP in the link below. Note: `custom_dict` will accept Trie structures as `Trie(iterable)`.

https://pythainlp.org/docs/5.0/api/tokenize.html#pythainlp.tokenize.word_tokenize

In [40]:
####FILL CODE HERE####
print("longest")
print(word_tokenize(input_text, Trie(thai_vocab), engine='longest'))
print("newmm")
print(word_tokenize(input_text, Trie(thai_vocab), engine='newmm'))
######################

longest
['ไป', 'หาม', 'เห', 'สี', '!']
newmm
['ไป', 'หา', 'มเหสี', '!']


## Part 2) Maximal Matching from Scratch

### Maximal matching
Complete the maximal matching function below with dynamic programming to tokenize the input text and output the 2D numerical array shown in class.

In [47]:
from math import inf #infinity
trie = Trie(thai_vocab)
def maximal_matching(c):
    #Initialize an empty 2D list
    d = [[None]*len(c) for _ in range(len(c))]

    ####FILL CODE HERE####
    n = len(c)
    for j in range(n):
        if c[: j+1] in thai_vocab:
            d[0][j] = 1
        else:
            d[0][j] = inf
    
    for i in range(1, n):
        for j in range(i, n):
            substring = c[i : j+1]
            if substring in thai_vocab:
                possible_vals = []
                for k in range(i):
                    if d[k][i-1] is not None and d[k][i-1] != inf:
                        possible_vals.append(d[k][i-1])
                
                if possible_vals:
                    d[i][j] = 1 + min(possible_vals)
                else:
                    d[i][j] = inf
            else:
                d[i][j] = inf
    ######################

    return d


maximal_matching(input_text)

[[1, 1, inf, inf, inf, inf, inf, inf, inf, inf],
 [None, 2, inf, inf, inf, inf, inf, inf, inf, inf],
 [None, None, 2, 2, 2, inf, inf, inf, inf, inf],
 [None, None, None, 3, inf, inf, inf, inf, inf, inf],
 [None, None, None, None, 3, inf, inf, inf, 3, inf],
 [None, None, None, None, None, 3, 3, inf, inf, inf],
 [None, None, None, None, None, None, 4, inf, inf, inf],
 [None, None, None, None, None, None, None, 4, 4, inf],
 [None, None, None, None, None, None, None, None, 5, inf],
 [None, None, None, None, None, None, None, None, None, 4]]

### Backtracking
Complete the backtracking function below to find the tokenzied words.
It should return a list containing a pair of the beginning position and the ending position of each word.
In this example, it should return:
<br>
[(0, 1),(2, 3),(4, 8),(9, 9)]
<br>
#### Each pair contains the position of each word as follows:
(0, 1) ไป
<br>
(2, 3) หา
<br>
(4, 8) มเหสี
<br>
(9, 9) !


In [51]:
def backtrack(d):
    eow = len(d)-1 # End of Word position
    word_pos = [] # Word position
    ####FILL CODE HERE####
    while eow >= 0:
        best_row = None
        best_val = float('inf')
        # Find the best row
        for i in range(eow, -1, -1):
            val = d[i][eow]
            if val is not None and val < best_val:
                best_val = val
                best_row = i

        # Record the word position
        word_pos.append((best_row, eow))
        # Move to the next end of word
        eow = best_row - 1
     ######################
    word_pos.reverse()
    return word_pos

# -- test
rs = backtrack(maximal_matching(input_text))
# Get word
for s,e in rs:
    print(s,e, input_text[s:e+1])

0 1 ไป
2 3 หา
4 8 มเหสี
9 9 !


### Test your maximal matching algorithm on a toy dictionary

Expected output:

[1, 1, inf, inf, inf, inf, inf, inf, inf, inf] ไ
<br>
[None, 2, inf, inf, inf, inf, inf, inf, inf, inf] ป
<br>
[None, None, 2, 2, 2, inf, inf, inf, inf, inf] ห
<br>
[None, None, None, 3, inf, inf, inf, inf, inf, inf] า
<br>
[None, None, None, None, 3, inf, inf, inf, 3, inf] ม
<br>
[None, None, None, None, None, 3, 3, inf, inf, inf] เ
<br>
[None, None, None, None, None, None, 4, inf, inf, inf] ห
<br>
[None, None, None, None, None, None, None, 4, 4, inf] ส
<br>
[None, None, None, None, None, None, None, None, 5, inf] ี
<br>
[None, None, None, None, None, None, None, None, None, 4] !
<br>

In [52]:
input_text = "ไปหามเหสี!"
out = maximal_matching(input_text)
for i in range(len(out)):
    print(out[i],input_text[i])

[1, 1, inf, inf, inf, inf, inf, inf, inf, inf] ไ
[None, 2, inf, inf, inf, inf, inf, inf, inf, inf] ป
[None, None, 2, 2, 2, inf, inf, inf, inf, inf] ห
[None, None, None, 3, inf, inf, inf, inf, inf, inf] า
[None, None, None, None, 3, inf, inf, inf, 3, inf] ม
[None, None, None, None, None, 3, 3, inf, inf, inf] เ
[None, None, None, None, None, None, 4, inf, inf, inf] ห
[None, None, None, None, None, None, None, 4, 4, inf] ส
[None, None, None, None, None, None, None, None, 5, inf] ี
[None, None, None, None, None, None, None, None, None, 4] !


### Test your backtracking algorithm on a toy dictionary
Compare your results with the result from PyThaiNLP `newmm`.

Expected output:
<br>
ไป|หา|มเหสี|!

In [60]:
def print_tokenized_text(d, input_text):
    tokenized_text=[]
    for pos in backtrack(d):
        #print(pos)
        tokenized_text.append(input_text[pos[0]:pos[1]+1])

    print("|".join(tokenized_text))

def get_list_of_words(d, input_text):
    tokenized_text=[]
    for pos in backtrack(d):
        tokenized_text.append(input_text[pos[0]:pos[1]+1])
    return tokenized_text

print_tokenized_text(out,input_text)
print(get_list_of_words(out,input_text))

ไป|หา|หมามเ|ห
['ไป', 'หา', 'หมามเ', 'ห']


### <font color=blue>Question 1</font>
Using your maximal matching code with the toy dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #1 in MyCourseVille. Also print out the answer in this notebook as well.

In [61]:
input_text = "ไปหาหมามเหสีมาหาม!"
low = get_list_of_words(maximal_matching(input_text),input_text)
print(len(low))
print_tokenized_text(maximal_matching(input_text),input_text)

10
ไป|หา|ห|ม|า|มเหสี|ม|า|หาม|!


## Part 3) Your Maximal Matching with Real Dictionary

For UNIX-based OS users, the following cell will download a dictionary (it's just a list of thai words). Alternatively, you can download it from this link: https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

In [62]:
!wget https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

--2025-01-19 17:52:58--  https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1519661 (1.4M) [text/plain]
Saving to: ‘words_th.txt’

words_th.txt        100%[===================>]   1.45M  5.11MB/s    in 0.3s    

2025-01-19 17:52:59 (5.11 MB/s) - ‘words_th.txt’ saved [1519661/1519661]



In [63]:
with open("words_th.txt",encoding='utf-8-sig') as f:
    thai_vocab = f.read().splitlines()
print("Vocab size:", len(thai_vocab))
print(thai_vocab[:10])

thai_vocab.extend(["ๆ","!"])

Vocab size: 62080
['ก ข ไม่กระดิกหู', 'ก.', 'ก.ค.', 'ก.ต.', 'ก.ป.ส.', 'ก.พ.', 'ก.พ.ด.', 'ก.ม.', 'ก.ย.', 'ก.ย']


### Part 3.1) The output of **YOUR** maximal matching algoithm on the new dictionary
Expected output:
<br>
[inf, 1, inf, 1, inf, inf, inf, inf, inf] ไ
<br>
[None, inf, inf, inf, inf, inf, inf, inf, inf] ป
<br>
[None, None, inf, 2, 2, inf, inf, inf, inf] ห
<br>
[None, None, None, inf, inf, inf, inf, inf, inf] า
<br>
[None, None, None, None, inf, inf, inf, inf, 2] ม
<br>
[None, None, None, None, None, inf, 3, inf, inf] เ
<br>
[None, None, None, None, None, None, inf, inf, inf] ห
<br>
[None, None, None, None, None, None, None, inf, 4] ส
<br>
[None, None, None, None, None, None, None, None, inf] ี

### Expected tokenized text
ไปหา|มเหสี

_Question: Why are the resulting tokens different?_

In [64]:
input_text = "ไปหามเหสี"
out = maximal_matching(input_text)
for i in range(len(out)):
    print(out[i],input_text[i])

print_tokenized_text(out,input_text)

[inf, 1, inf, 1, inf, inf, inf, inf, inf] ไ
[None, inf, inf, inf, inf, inf, inf, inf, inf] ป
[None, None, inf, 2, 2, inf, inf, inf, inf] ห
[None, None, None, inf, inf, inf, inf, inf, inf] า
[None, None, None, None, inf, inf, inf, inf, 2] ม
[None, None, None, None, None, inf, 3, inf, inf] เ
[None, None, None, None, None, None, inf, inf, inf] ห
[None, None, None, None, None, None, None, inf, 4] ส
[None, None, None, None, None, None, None, None, inf] ี
ไปหา|มเหสี


### <font color=blue>Question 2</font>
Using your maximal matching algorithm and the actual Thai dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #2 in MyCourseVille. Also print out the answer in this notebook as well.

In [67]:
input_text = "ประเทศไทยรวมเลือดเนื้อชาติเชื้อไทยเป็นประชารัฐไผทของไทยทุกส่วนอยู่ดำรงคงไว้ได้ทั้งมวลด้วยไทยล้วนหมายรักสามัคคี"

print(len(get_list_of_words(maximal_matching(input_text),input_text)))
print_tokenized_text(maximal_matching(input_text),input_text)

26
ประเทศ|ไทย|รวม|เลือดเนื้อ|ชาติ|เชื้อ|ไทย|เป็น|ประชา|รัฐ|ไผท|ของ|ไทย|ทุก|ส่วน|อยู่|ดำรง|คงไว้|ได้|ทั้งมวล|ด้วย|ไทย|ล้วน|หมาย|รัก|สามัคคี


### Part 3.2) Use PyThaiNLP `word_tokenize` with custom dictionary

Try tokenizing the following text with `word_tokenize` in `newmm` algorithm and default real dictionary.

In [68]:
text='นัดกินกันตอนไหนก็ได้ที่สามย่านมิตรทาวน์'

####FILL CODE HERE####
tokenized = word_tokenize(text, Trie(thai_vocab), engine='newmm')
print(tokenized)
######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่าน', 'มิตร', 'ทาวน์']


Add 'สามย่านมิตรทาวน์' into dictionary and then tokenize again

In [70]:
####FILL CODE HERE####
print(word_tokenize(text, Trie(thai_vocab + ["สามย่านมิตรทาวน์"]), engine='newmm'))
######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่านมิตรทาวน์']


### <font color=blue>Question 3</font>
Using the code from part three only, how many “words” did you get when tokenizing this input text **after adding the new vocabs**.

Answer this question in question #3 in MyCourseVille. Also print out the answer in this notebook as well.

In [72]:
new_vocab = ["ดิสนีย์ออนไอซ์", "ตีกอล์ฟ", "ธรรมมะ"]
input_text = "อ๋อก็ว่าจะไปเรียนแต่งหน้านั่งสมาธิดำน้ำปลูกปะการังทำอาหารนวดสปาปลูกป่าดำนาดูดิสนีย์ออนไอซ์แรลลี่ตีกอล์ฟล่องเรือส่องสัตว์ช้อปปิ้งดูงิ้วดูละครเวทีดูคอนเสิร์ตดินเนอร์ทำขนมจัดดอกไม้เที่ยวตลาดน้ำเรียนถ่ายรูปดูกายกรรมชมเมืองเก่าเข้าสัมมนาทัวร์ธรรมมะเรียนเต้นแล้วก็ร้องเพลง"

rs = word_tokenize(input_text, Trie(thai_vocab + new_vocab), engine='newmm')
print(len(rs))
print(rs)

51
['อ๋อ', 'ก็', 'ว่า', 'จะ', 'ไป', 'เรียน', 'แต่งหน้า', 'นั่งสมาธิ', 'ดำน้ำ', 'ปลูก', 'ปะการัง', 'ทำอาหาร', 'นวด', 'สปา', 'ปลูกป่า', 'ดำนา', 'ดู', 'ดิสนีย์ออนไอซ์', 'แรลลี่', 'ตีกอล์ฟ', 'ล่องเรือ', 'ส่องสัตว์', 'ช้อปปิ้ง', 'ดู', 'งิ้ว', 'ดู', 'ละครเวที', 'ดู', 'คอนเสิร์ต', 'ดินเนอร์', 'ทำ', 'ขนม', 'จัด', 'ดอกไม้', 'เที่ยว', 'ตลาดน้ำ', 'เรียน', 'ถ่ายรูป', 'ดู', 'กายกรรม', 'ชม', 'เมือง', 'เก่า', 'เข้า', 'สัมมนา', 'ทัวร์', 'ธรรมมะ', 'เรียน', 'เต้น', 'แล้วก็', 'ร้องเพลง']


## Part 4) Use maximal matching on real dataset

To complete this exercise, we will use the maximal matching algorithm on NECTEC's BEST corpus.

The corpus has a structure of characters with target whether it's a beginning of a word (True/False).

In [74]:
#Download dataset
!uv add gdown
!gdown "1EcrlXYUyIEM3aeIJse6nPpiv_UjSKgoU&confirm=t"

Resolved 78 packages in 1.37s                                        
⠙ Preparing packages... (0/4)                                                   
⠙ Preparing packages... (0/4)----     0 B/36.19 kB                      
⠙ Preparing packages... (0/4)---- 4.13 kB/36.19 kB                      
⠙ Preparing packages... (0/4)---- 8.27 kB/36.19 kB                      
⠙ Preparing packages... (0/4)---- 12.40 kB/36.19 kB                     
pysocks    ------------------------------     0 B/16.73 kB
⠙ Preparing packages... (0/4)---- 12.40 kB/36.19 kB                     
pysocks    ------------------------------ 2.76 kB/16.73 kB
⠙ Preparing packages... (0/4)---- 12.40 kB/36.19 kB                     
pysocks    ------------------------------ 4.13 kB/16.73 kB
⠙ Preparing packages... (0/4)---- 12.40 kB/36.19 kB                     
pysocks    ------------------------------ 6.89 kB/16.73 kB
⠙ Preparing packages... (0/4)---- 12.40 kB/36.19 kB                     
pysocks    ------------------

In [75]:
!tar xvf corpora.tar.gz

x corpora/
x corpora/mnist_data/
x corpora/mnist_data/t10k-images-idx3-ubyte.gz
x corpora/mnist_data/train-images-idx3-ubyte.gz
x corpora/mnist_data/.ipynb_checkpoints/
x corpora/mnist_data/vis_utils.py
x corpora/mnist_data/__init__.py
x corpora/mnist_data/load_mnist.py
x corpora/mnist_data/train-labels-idx1-ubyte.gz
x corpora/mnist_data/t10k-labels-idx1-ubyte.gz
x corpora/BEST/
x corpora/BEST/test/
x corpora/BEST/test/df_best_article_test.csv
x corpora/BEST/test/df_best_encyclopedia_test.csv
x corpora/BEST/test/df_best_novel_test.csv
x corpora/BEST/test/df_best_news_test.csv
x corpora/BEST/train/
x corpora/BEST/train/df_best_encyclopedia_train.csv
x corpora/BEST/train/df_best_article_train.csv
x corpora/BEST/train/df_best_news_train.csv
x corpora/BEST/train/df_best_novel_train.csv
x corpora/BEST/val/
x corpora/BEST/val/df_best_encyclopedia_val.csv
x corpora/BEST/val/df_best_news_val.csv
x corpora/BEST/val/df_best_article_val.csv
x corpora/BEST/val/df_best_novel_val.csv
x corpora/.ipyn

In [76]:
!uv add pandas
import pandas as pd
import os

Resolved 80 packages in 165ms                                        
Installed 3 packages in 64ms                                     ░░░░░░░░░░░░░░░░░░░░ [0/0] Installing wheels...                                 
 + pandas==2.2.3
 + pytz==2024.2
 + tzdata==2024.2


In [77]:
# Path to the preprocessed data
best_processed_path = 'corpora/BEST'
option = "test"

df = []
# article types in BEST corpus
article_types = ['article', 'encyclopedia', 'news', 'novel']
for article_type in article_types:
    df.append(pd.read_csv(os.path.join(best_processed_path, option, 'df_best_{}_{}.csv'.format(article_type, option))))
df = pd.concat(df)
df

,char,target
0,ป,True
1,ฏ,False
2,ิ,False
3,ร,False
4,ู,False
...,...,...
644911,ห,False
644912,น,False
644913,ม,True
644914,า,False


In [78]:
len(df)

2271932

In [79]:
# Some text in this corpus
all_text = "".join(df['char'].tolist())
all_text[:1000]

'ปฏิรูปการศึกษา : มุมมองทางกระบวนทัศน์และบริบทสังคมไทยThe Reformation of Eucation from A Thai Perspectiveกระบวนทัศน์และวิธีคิดแบบแยกส่วน ลดส่วน ได้ทำให้"การศึกษาเรียนรู้"ใน หลายทศวรรษที่ผ่านมา กลายเป็นเรื่องของนักวิชาการด้านศึกษาศาสตร์ ครุศาสตร์ หรือเป็นเรื่องของโรงเรียน ครูอาจารย์ กระทรวงศึกษาธิการ ทบวงมหาวิทยาลัยฯ มาอย่างต่อเนื่องยาวนาน (เหมือนกับที่เรื่องสุขภาพเป็นเรื่องของแพทย์และโรงพยาบาล) การจัดการศึกษาภายใต้กระบวนทัศน์และวิธีคิดแบบดังกล่าวของรัฐ ได้ถูกวิพากษ์วิจารณ์และตกเป็นจำเลยจากวิกฤตการณ์ทางสังคมมากมาย อันสะท้อนถึงความล้มเหลวของการจัดการศึกษาเพื่อพัฒนามนุษย์ (ปัญหาศีลธรรมเสื่อมถอย ยาเสพติด การขาดจิตสำนึกทางสังคม ฯลฯ) ซึ่งสังคมร่วมกันสรุปว่า เกิดจากความล้มเหลวของระบบการศึกษาในกระบวนทัศน์แบบแยกส่วน นำมาสู่การปฏิรูปการศึกษาที่กำลังดำเนินการอยู่ในปัจจุบัน ด้วยเป้าหมายเพื่อสร้างการเรียนรู้แบบองค์รวม ที่จะทำให้"ผู้เรียนเก่ง-ดี-มีความสุข"คำถามที่ผู้เขียนสนใจในการปฏิรูปการศึกษาที่ดำเนินการในปัจจุบัน คือ๑.การปฏิรูปการศึกษาในปัจจุบันดำเนินการภายใต้กระบวนทัศน์แบบบูรณาการ (องค์รวม)ตามที

### <font color=blue>Question 4</font>
Using PyThaiNLP `newmm`, how many words did you get in the BEST corpus (test)? [Runtime is around 7 mins] What are the accuracy, f1, precision, recall scores for each character?

Answer this question in question #4 in MyCourseVille. Also print out the answer in this notebook as well.

_Question: What main metric should we look at? Why?_

In [80]:
!uv add scikit-learn

⠹ threadpoolctl==3.5.0                                                          Resolved 84 packages in 330ms
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)------     0 B/11.12 MB                    
⠙ Preparing packages... (0/2)------ 2.76 kB/11.12 MB                    
⠙ Preparing packages... (0/2)------ 6.89 kB/11.12 MB                    
⠙ Preparing packages... (0/2)------ 9.65 kB/11.12 MB                    
⠙ Preparing packages... (0/2)------ 12.40 kB/11.12 MB                   
⠙ Preparing packages... (0/2)------ 15.16 kB/11.12 MB                   
⠙ Preparing packages... (0/2)------ 19.14 kB/11.12 MB                   
⠙ Preparing packages... (0/2)------ 23.27 kB/11.12 MB                   
scikit-learn ------------------------------ 23.27 kB/11.12 MB
⠙ Preparing packages... (0/2)----     0 B/24.81 MB                      
scikit-learn ------------------------------ 27.41 kB/11.12 MB
⠙ Preparing packages... (0/2

In [83]:
####FILL CODE HERE####
import pandas as pd
import os
from pythainlp.tokenize import word_tokenize

text = "".join(df["char"].values.tolist())
tokens = word_tokenize(text, engine='newmm')
print(f"Number of tokens from newmm = {len(tokens):,d}")

Number of tokens from newmm = 569,631


In [84]:
def convert_to_character(_tokens):
  char_list = [0]*len("".join(_tokens))
  char_count = 0
  for word in _tokens:
    char_list[char_count] = 1
    char_count += len(word)
  return char_list

# chars = convert_to_character(tokens)
y_pred = convert_to_character(tokens)
y_pred[:20]

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]

In [85]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

####FILL CODE HERE####
y_true = df["target"].astype(int).values
print(f"Length of y_true = {len(y_true):,d} characters")

acc = accuracy_score(y_true, y_pred)
p = precision_score(y_true, y_pred, pos_label=1)
r = recall_score(y_true, y_pred, pos_label=1)
f1 = f1_score(y_true, y_pred, pos_label=1)

print("===== Character-Level Metrics =====")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {p:.4f}")
print(f"Recall   : {r:.4f}")
print(f"F1 Score : {f1:.4f}")
######################

Length of y_true = 2,271,932 characters
===== Character-Level Metrics =====
Accuracy : 0.9431
Precision: 0.9423
Recall   : 0.8479
F1 Score : 0.8926
